# PlayWright Browser Toolkit

This toolkit is used to interact with the browser. While other tools (like the `Requests` tools) are fine for static sites, `PlayWright Browser` toolkits let your agent navigate the web and interact with dynamically rendered sites. 

Some tools bundled within the `PlayWright Browser` toolkit include:

- `NavigateTool` (navigate_browser) - navigate to a URL
- `NavigateBackTool` (previous_page) - wait for an element to appear
- `ClickTool` (click_element) - click on an element (specified by selector)
- `ExtractTextTool` (extract_text) - use beautiful soup to extract text from the current web page
- `ExtractHyperlinksTool` (extract_hyperlinks) - use beautiful soup to extract hyperlinks from the current web page
- `GetElementsTool` (get_elements) - select elements by CSS selector
- `CurrentPageTool` (current_page) - get the current page URL


In [1]:
%pip install --upgrade --quiet  playwright > /dev/null
%pip install --upgrade --quiet  lxml

# If this is your first time using playwright, you'll have to install a browser executable.
# Running `playwright install` by default installs a chromium browser executable.
# playwright install


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit

Async function to create context and launch browser:

In [2]:
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.\n",	  },
)

In [3]:
# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply()

## Instantiating a Browser Toolkit

It's always recommended to instantiate using the `from_browser` method so that the 

In [4]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/isaachershenson/Library/Caches/ms-playwright/chromium-1124/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=127.0.6533.17>),
 NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/isaachershenson/Library/Caches/ms-playwright/chromium-1124/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=127.0.6533.17>),
 NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/isaachershenson/Library/Caches/ms-playwright/chromium-1124/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=127.0.6533.17>),
 ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/isaachershenson/Library/Caches/ms-playwright/chromium-1124/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=127.0.6533.17>),
 ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/U

In [5]:
tools_by_name = {tool.name: tool for tool in tools}
navigate_tool = tools_by_name["navigate_browser"]
get_elements_tool = tools_by_name["get_elements"]

In [6]:
await navigate_tool.arun(
    {"url": "https://web.archive.org/web/20230428133211/https://cnn.com/world"}
)

'Navigating to https://web.archive.org/web/20230428133211/https://cnn.com/world returned status code 200'

In [7]:
# The browser is shared across tools, so the agent can interact in a stateful manner
await get_elements_tool.arun(
    {"selector": ".container__headline", "attributes": ["innerText"]}
)

'[{"innerText": "These Ukrainian veterinarians are risking their lives to care for dogs and cats in the war zone"}, {"innerText": "Life in the ocean’s ‘twilight zone’ could disappear due to the climate crisis"}, {"innerText": "Clashes renew in West Darfur as food and water shortages worsen in Sudan violence"}, {"innerText": "Thai policeman’s wife investigated over alleged murder and a dozen other poison cases"}, {"innerText": "American teacher escaped Sudan on French evacuation plane, with no help offered back home"}, {"innerText": "Dubai’s emerging hip-hop scene is finding its voice"}, {"innerText": "How an underwater film inspired a marine protected area off Kenya’s coast"}, {"innerText": "The Iranian drones deployed by Russia in Ukraine are powered by stolen Western technology, research reveals"}, {"innerText": "India says border violations erode ‘entire basis’ of ties with China"}, {"innerText": "Australian police sift through 3,000 tons of trash for missing woman’s remains"}, {"in

In [8]:
# If the agent wants to remember the current webpage, it can use the `current_webpage` tool
await tools_by_name["current_webpage"].arun({})

'https://web.archive.org/web/20230428133211/https://cnn.com/world'

## Use within an Agent

Several of the browser tools are `StructuredTool`'s, meaning they expect multiple arguments. These aren't compatible (out of the box) with agents older than the `STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION`

In [10]:
from langchain.agents import AgentType, initialize_agent
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
    model_name="claude-3-haiku-20240307", temperature=0
)  # or any other LLM, e.g., ChatOpenAI(), OpenAI()

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

/Users/isaachershenson/.pyenv/versions/3.11.9/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [11]:
result = await agent_chain.arun("What are the headers on langchain.com?")
print(result)



> Entering new AgentExecutor chain...
Thought: To find the headers on langchain.com, I will navigate to the website and extract the text.

Action:
```
{
  "action": "navigate_browser",
  "action_input": "https://langchain.com"
}
```


Observation: Navigating to https://langchain.com returned status code 200
Thought:Okay, let's find the headers on the langchain.com website.

Action:
```
{
  "action": "extract_text",
  "action_input": {}
}
```


Observation: LangChain We value your privacy We use cookies to analyze our traffic. By clicking "Accept All", you consent to our use of cookies. Privacy Policy Customize Reject All Accept All Customize Consent Preferences We may use cookies to help you navigate efficiently and perform certain functions. You will find detailed information about all cookies under each consent category below. The cookies that are categorized as "Necessary" are stored on your browser as they are essential for enabling the basic functionalities of the site.... Show 